In [2]:
import torchaudio
from torchaudio.datasets import LIBRISPEECH

dataset = LIBRISPEECH(root='../input', url='test-clean', download=False)

In [ ]:
""" waveform, sample_rate, transcript, _, _, _ = dataset[0]

print("Waveform shape:", waveform.shape) 
print("Sample rate:", sample_rate)
print("Transcript:", transcript) """

NameError: name 'dataset' is not defined

In [7]:
import torchaudio.transforms as T
import torch

In [ ]:
""" TARGET_SAMPLE_RATE = 16000
TARGET_NUM_CHANNELS = 2
TARGET_DURATION = 5 
TARGET_NUM_SAMPLES = TARGET_SAMPLE_RATE * TARGET_DURATION """

def preprocess_waveform(waveform, sample_rate):
    """
    Resample output for original sample to 16000
    Convert to stereo
    pad or truncate th fixed duration
    Input shape: [channels, num_samples]
    Output shape: [2, TARGET_NUM_SAMPLES]
    """
    if waveform.shape[0] == 1:
        waveform = waveform.repeat(TARGET_NUM_CHANNELS, 1)

    if sample_rate != TARGET_SAMPLE_RATE:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=TARGET_SAMPLE_RATE)
        waveform = torch.stack([
            resampler(wf.unsqueeze(0)).squeeze(0)
            for wf in waveform
        ])
    
    if waveform.shape[1] > TARGET_NUM_SAMPLES:
        waveform = waveform[:,:TARGET_NUM_SAMPLES]
    elif waveform.shape[1] < TARGET_NUM_SAMPLES:
        pad_length = TARGET_NUM_SAMPLES - waveform.shape[1]
        waveform = torch.nn.functional.pad(waveform, (0, pad_length))  
    return waveform  
    

In [ ]:
import random
def time_shift(waveform, shift_limit=0.2):
    """moves the entire audio forward or backward slightly in time"""
    shift_amt = int(random.uniform(-shift_limit, shift_limit) * waveform.size(1))
    return torch.roll(waveform, shifts=shift_amt, dims=1)
def speed_change(waveform, rate):
    """makes the audio play faster or slower"""
    effects = [['speed', str(rate)], ['rate', '16000']]
    waveform, _ = torchaudio.sox_effects.apply_effects_tensor(waveform, 16000, effects)
    return waveform
def pitch_shift(waveform, n_steps):
    """changes how high or low the speaker sounds"""
    effects = [['pitch', str(n_steps * 100)], ['rate', '16000']]
    waveform, _ = torchaudio.sox_effects.apply_effects_tensor(waveform, 16000, effects)
    return waveform

In [ ]:
mel_transform = T.MelSpectrogram(
    sample_rate=16000,
    n_fft=1024,
    hop_length=512,
    n_mels=80,
)

In [10]:
mfcc_transform = T.MFCC(
    sample_rate=16000,
    n_mfcc=13,
    melkwargs={"n_fft": 1024, "hop_length": 512, "n_mels": 80}
)